In [ ]:
# notebook to produce he2149 lightcurve plots:

import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

In [ ]:
# load results:

def diffdata(file):
    time,magA,magA_err,magB,magB_err,sky,residue,chi2red = \
    np.loadtxt(file,skiprows=4,usecols=(1,7,8,9,10,11,13,14),unpack=True)
    return time,magA,magA_err,magB,magB_err,sky,residue,chi2red
def refdata(file):
    time,magA,magA_err,magB,magB_err,sky,residue,chi2red = \
    np.loadtxt(file,skiprows=2,max_rows=1,usecols=(1,7,8,9,10,11,13,14),unpack=True)
    return time,magA,magA_err,magB,magB_err,sky,residue,chi2red

data_file_name = 'diffgalfitresults_betterpos.txt'
R_all = diffdata('R/diff_galfit/'+data_file_name)
R_ref = refdata('R/diff_galfit/'+data_file_name)
V_all = diffdata('V/diff_galfit/'+data_file_name)
V_ref = refdata('V/diff_galfit/'+data_file_name)

In [ ]:
# flux to magnitues:

def mag(alldata,refdata,offset):
    time     = alldata[0]
    magA_err = 2.5/np.log(10)*np.sqrt(alldata[2]**2+refdata[2]**2)/(alldata[1]+refdata[1])
    magB_err = 2.5/np.log(10)*np.sqrt(alldata[4]**2+refdata[4]**2)/(alldata[3]+refdata[3])
    magA     = -2.5*np.log10(alldata[1]+refdata[1]) + offset
    magB     = -2.5*np.log10(alldata[3]+refdata[3]) + offset
    return time,magA,magA_err,magB,magB_err

# Offset (determined from reference stars with known apperent GAIA magnitudes in the reference image):
R_offset  = 27.024
V_offset  = 29.209

R_mag = mag(R_all,R_ref,R_offset)
V_mag = mag(V_all,V_ref,V_offset)

In [ ]:
def plot(mag,offsetA=0.0,offsetB=0.0,lable1='',lable2='',sym1='o',sym2='o',timedelay=0.0):
    plt.errorbar(mag[0],mag[1]+offsetA,yerr=mag[2],fmt=sym1,markersize=4,elinewidth=1,capsize=3,label=lable1)
    plt.errorbar(mag[0]+timedelay,mag[3]+offsetB,yerr=mag[4],fmt=sym2,markersize=4,elinewidth=1,capsize=3,label=lable2)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of Q2237+0305 in $R$ and $V$ band',size=17)
plot(R_mag,0.0,-0.5,'$R$ band: $A$','$R$ band: $B-0.5$',timedelay=-103.0/365.25)
plot(V_mag,0.0,-0.5,'$V$ band: $A$','$V$ band: $B-0.5$',timedelay=-103.0/365.25)
plt.gca().invert_yaxis()
plt.xlabel('detection time in years (time delays not included)',size=12)
plt.ylabel('apparent magnitudes',size=12)
plt.legend(fontsize=8,loc='lower left',ncol=4)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag,0.0,-1.16,'$R$ band: $A$','$R$ band: $B-1.16$','o','^',-103.0/365.25)
plot(V_mag,0.2,-0.89,'$V$ band: $A+0.2$','$V$ band: $B-0.89$','s','v',-103.0/365.25)
plt.ylim([16.51,17.69])
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,loc='upper left',ncol=2)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
plt.ylim([16.51,17.69])
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,loc='upper left',ncol=2)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,loc='upper left',ncol=2)

In [ ]:
def mask(alldata,refdata):
    sky     = alldata[5]
    residue = alldata[6]
    chi2red = alldata[7]
    
    fluxA   = alldata[1]+refdata[1]
    fluxAer = alldata[2]
    fluxB   = alldata[3]+refdata[3]
    fluxBer = alldata[4]
    #print(np.abs(fluxA/fluxAer),np.abs(fluxB/fluxBer))
    
    max_chi = 5.0
    max_sky = 10.0
    max_res = 15.0 #20.0
    
    #mask = np.abs(np.where(chi2red>max_chi,0,1)*np.where(sky>max_sky,0,1)*np.where(residue>max_res,0,1)-1)
    
    mask = np.abs(np.where(np.abs(fluxAer/fluxA)>0.02,0,1)-1)
    keep=mask==1
    index=np.arange(len(keep))    
    print (index[keep],(fluxAer/fluxA)[keep])
    
    #print(index[keep])
    #print ((sky/sky_err)[keep])
    
    only_chi2red_mask = False
    if only_chi2red_mask:
        max_chi = 5.0
        min_chi = 0.2
        mask = np.abs(np.where(chi2red>max_chi,0,1)*np.where(chi2red<min_chi,0,1)-1)
        
    chi2red_sky_mask = False
    if chi2red_sky_mask:
        max_chi = 4.0
        min_chi = 0.5
        max_sky = 5.0
        mask = np.abs(np.where(chi2red>max_chi,0,1)*np.where(chi2red<min_chi,0,1)*np.where(sky>max_sky,0,1)-1)
    
    justchi = False
    if justchi:
        max_chi = 5.0
        mask = np.abs(np.where(chi2red>max_chi,0,1)-1)  
    
    return mask

R_mask = mask(R_all,R_ref)
V_mask = mask(V_all,V_ref)
print(R_mask.sum(),V_mask.sum())

In [ ]:
def masked_mag(mag,mask):
    time = np.ma.compressed(np.ma.array(mag[0],mask=mask))
    magA = np.ma.compressed(np.ma.array(mag[1],mask=mask))
    magA_err = np.ma.compressed(np.ma.array(mag[2],mask=mask))
    magB = np.ma.compressed(np.ma.array(mag[3],mask=mask))
    magB_err = np.ma.compressed(np.ma.array(mag[4],mask=mask))
    return time,magA,magA_err,magB,magB_err

R_mag_masked = masked_mag(R_mag,R_mask)
V_mag_masked = masked_mag(V_mag,V_mask)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag_masked,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag_masked,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
#plt.ylim([16.51,17.69])
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,ncol=2)

In [ ]:
def newmask(alldata,refdata):
    fluxA   = alldata[1]+refdata[1]
    fluxAer = np.sqrt(alldata[2]*alldata[2]+refdata[2]*refdata[2])
    fluxB   = alldata[3]+refdata[3]
    fluxBer = np.sqrt(alldata[4]*alldata[4]+refdata[4]*refdata[4])
    sky     = alldata[5]
    residue = alldata[6]
    chi2red = alldata[7]

    max_rel_flux_err = 0.07
    mrfe = max_rel_flux_err
    
    mask = np.abs(np.where(np.abs(fluxAer/fluxA)>mrfe,0,1)*np.where(np.abs(fluxBer/fluxB)>mrfe,0,1)-1)
    keep=mask==1
    index=np.arange(len(keep))    
    print('file index',index[keep])
    print('flux A/err',(fluxAer/fluxA)[keep])
    print('flux B/err',(fluxBer/fluxB)[keep])
    print('background',sky[keep])
    print('residue   ',residue[keep])
    print('chr2red   ',chi2red[keep])
    
    return mask

R_newmask = newmask(R_all,R_ref)
print('')
V_newmask = newmask(V_all,V_ref)
print('')
print('masked in R:',R_newmask.sum())
print('masked in V:',V_newmask.sum())

R_mag_newmasked = masked_mag(R_mag,R_newmask)
V_mag_newmasked = masked_mag(V_mag,V_newmask)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag,0.0,-1.2,'$R$: $A$ not masked','$R$: $B-1.2$ not masked','o','^',-32.4/365.25)
plot(V_mag,0.2,-1.0,'$V$: $A+0.2$ not masked','$V$: $B-1.0$ not masked','s','v',-32.4/365.25)
plot(R_mag_newmasked,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag_newmasked,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,ncol=2)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag_newmasked,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag_newmasked,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,ncol=2)

In [ ]:
max_mag_err = np.max([np.max(R_mag[2]),np.max(R_mag[4]),np.max(V_mag[2]),np.max(V_mag[4])])
print(max_mag_err)
bins = np.linspace(0.0,max_mag_err,num=100,endpoint=True)
plt.hist(R_mag[2],bins,histtype='step',label='$\Delta m_{R,A}$')
plt.hist(R_mag[4],bins,histtype='step',label='$\Delta m_{R,B}$')
plt.hist(V_mag[2],bins,histtype='step',label='$\Delta m_{V,A}$')
plt.hist(V_mag[4],bins,histtype='step',label='$\Delta m_{V,B}$')
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
def finalmask(lightcurve):
    mag_A_err = lightcurve[2]
    mag_B_err = lightcurve[4]
    
    max_mag_err = 0.05
    
    mask = np.abs(np.where(np.abs(mag_A_err)>max_mag_err,0,1)*np.where(np.abs(mag_B_err)>max_mag_err,0,1)-1)
    
    return mask

R_finalmask = finalmask(R_mag)
V_finalmask = finalmask(V_mag)
print('masked in R:',R_finalmask.sum())
print('masked in V:',V_finalmask.sum())

R_mag_finalmasked = masked_mag(R_mag,R_finalmask)
V_mag_finalmasked = masked_mag(V_mag,V_finalmask)

In [ ]:
plt.figure(figsize=(10,7),dpi=400)
plt.title('light curves of HE2149-2745 in the $R$ and $V$ band',size=18)
plot(R_mag_finalmasked,0.0,-1.2,'$R$ band: $A$','$R$ band: $B-1.2$','o','^',-32.4/365.25)
plot(V_mag_finalmasked,0.2,-1.0,'$V$ band: $A+0.2$','$V$ band: $B-1.0$','s','v',-32.4/365.25)
plt.gca().invert_yaxis()
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('time $t_A$ and $t_B+\Delta t_{AB}$ [years]',size=15)
plt.ylabel('apparent magnitudes [mag]',size=15)
plt.legend(fontsize=13,ncol=2)

In [ ]:
def magtxt(quasar,mag):
    txt = open("lightcurves_"+str(quasar)+".txt", "w")
    txt.write("time"+"\t"+"A-mag"+"\t"+"A-err"+"\t"+"B-mag"+"\t"+"B-err"+"\n")
    txt.write("\n")
    for i in range(len(mag[0])):
        txt.write(str(mag[0][i])+"\t"+str(mag[1][i])+"\t"+str(mag[2][i])+"\t"+str(mag[3][i])+"\t"+str(mag[4][i])+"\n")
    txt.close()

In [ ]:
magtxt('he2149R',R_mag_finalmasked)
magtxt('he2149V',V_mag_finalmasked)

In [ ]:
print(np.sort(R_mag_finalmasked[0]))

In [ ]:
import glob
from astropy.io import fits
from astropy.time import Time

def bettertime(band,file,mask,mag):
    
    table = open(band+'/diff_galfit/'+file,'r')
    lines = table.readlines()[4:]
    notmaskeddates = np.ma.compressed(np.ma.array(np.arange(len(lines)),mask=mask))
    times = np.zeros(len(notmaskeddates))
    k = 0
    for i in notmaskeddates:
        fitsname = (lines[i].split('.fits')[0].replace('diff_',''))
        fitsfiles = list(glob.glob(band+'/raw/'+fitsname+'*.fits'))
        fitstimes = np.zeros(len(fitsfiles))
        for j in range(len(fitsfiles)):
            data = fits.open(fitsfiles[j])
            fitstimes[j] = data[0].header['MJD-OBS']
        meanMJD = np.mean(fitstimes)
        times[k] = Time(meanMJD,format='mjd').decimalyear
        k = k + 1
    if len(mag[0]) != len(times):
        print('WARNING!!!')
    difftest = np.array([np.mean(times - mag[0]),np.max(np.abs(times - mag[0]))])
    print('mean and maximal local date calc. time to mjd calc. time in days:',difftest*365.25)
        
    return times

betterRtime = bettertime('R',data_file_name,R_finalmask,R_mag_finalmasked)
betterVtime = bettertime('V',data_file_name,V_finalmask,V_mag_finalmasked)
#print(betterRtime,betterVtime)

In [ ]:
def overwritetime(mag,bettertime):
    return bettertime,mag[1],mag[2],mag[3],mag[4]
    
R_mag_finalmaskedandtime = overwritetime(R_mag_finalmasked,betterRtime)
V_mag_finalmaskedandtime = overwritetime(V_mag_finalmasked,betterVtime)

for i in range(5):
    print(i)
    print(R_mag_finalmasked[i][0:5]-R_mag_finalmaskedandtime[i][0:5])
    print(V_mag_finalmasked[i][0:5]-V_mag_finalmaskedandtime[i][0:5])

In [ ]:
magtxt('he2149R',R_mag_finalmaskedandtime)
magtxt('he2149V',V_mag_finalmaskedandtime)